In [94]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import cufflinks as cf
import glob


cf.go_offline()

sns.set(style="darkgrid")

In [95]:
data_paths = glob.glob('../PostHog/data/*.csv')
frames = [pd.read_csv(path, names=['title', 'language', 'icon', 'id', 'timestamp']) for path in data_paths]
data = pd.concat(frames)
data = data.drop_duplicates(subset=['id'])
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['title'] = data['title'].str.strip()
data.head()
data.groupby(pd.Grouper(key='timestamp', axis=0, freq='1D'))['id'].count().iplot(kind='bar')

In [96]:
df = data.groupby(by=["icon", "title", "language"])[['id']].nunique()
print(len(df))
filtered = df[df['id'] >= 3]
print(len(filtered))
filtered = filtered.reset_index()
filtered.sort_values(by='id', ascending=False).head()

1394927
275722


,icon,title,language,id
2329,alarm.fill,Wake up!,en,185365
182080,moon.fill,Sleep Well!,en,83554
965,alarm.fill,Aufwachen!,de,45134
83808,custom.shower,Shower,en,34387
197686,noun.meal,Lunch,en,20525


In [97]:
df.reset_index().groupby(by=["id"])[["id"]].count().iplot(kind='bar')

In [98]:
filtered.groupby(by=["language"])['id'].count().sort_values(ascending=False).iplot(kind='bar')

In [99]:
import json

for language in filtered["language"].drop_duplicates():
    export = filtered[filtered["language"] == language][['icon', 'title']]
    export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
    export.to_csv(f'data/{language}.csv', index=False)
    print(language, len(export))
export = filtered[['icon', 'title']]
export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
export.to_csv(f'data/all.csv', index=False)


en 154609
fr 11836
de 35415
es 9295
it 7350
pt 4256
nb 1388
ro 323
nl 10967
zh 12170
th 4938
sv 2228
pl 5962
vi 1867
fi 527
ru 3823
ar 986
ja 4794
tr 612
ca 234
id 424
hr 168
uk 345
da 1059
hu 50
fa 30
ko 64
ur 2


In [100]:
export = filtered
export.rename(columns={'icon': 'label', 'title': 'text'}, inplace=True)
export = export[['label', 'text']]
export.to_json('user_icons_learn.json', orient="records")
